### Imports

In [279]:
import pandas as pd
import numpy as np

import yfinance as yf
from ta.momentum import rsi

from pyhhmm.gaussian import GaussianHMM

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

In [298]:
# Get the data for the BTC-USD and DXY
ticker1 ='BTC-USD'
ticker2 = 'DX-Y.NYB'

df = yf.download(ticker1)
df_dxy= yf.download(ticker2)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Feature Engineering

In [299]:
# Add DXY Close to BTC dataframe
df_dxy=df_dxy.loc[df_dxy.index >= '2014-09-17',"Close"]
df['DXY']=df_dxy
df.ffill(inplace=True)
df_close=df['Close']



In [300]:
# Check the head of the data
df.head(2)

,Open,High,Low,Close,Adj Close,Volume,DXY
Date,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,84.699997
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,84.320000


In [305]:
# Add Returns and range to the dataframe
df['lrets'] = np.log(df['Close']/df['Close'].shift(1))
df['lrets_prod'] = df['lrets'].cumsum().apply(np.exp)-1
df['range'] = df['High']/df['Low'] -1


In [307]:
# Check the head of the data
df.head(3)


,Open,High,Low,Close,Adj Close,Volume,DXY,lrets,lrets_prod,range
Date,,,,,,,,,,
2024-01-27,41815.625000,42195.632812,41431.281250,42120.054688,42120.054688,11422941934,103.470001,0.007224,91.099108,0.018449
2024-01-28,42126.125000,42797.175781,41696.910156,42035.593750,42035.593750,16858971687,103.470001,-0.002007,90.914427,0.026387
2024-01-29,42030.914062,43228.625000,41841.148438,43146.390625,43146.390625,20599599104,103.471001,0.026082,93.343279,0.033161
